# PEtab 2.0 import

**Note:** PEtab v2 support in AMICI is experimental and subject to change.

This notebook demonstrates how to import a [PEtab v2 problem](https://petab.readthedocs.io/en/latest/v2/documentation_data_format.html) for computing the objective function and gradient, or for simulating individual conditions using AMICI. We assume that you are already familiar with the basics of AMICI and PEtab.

In [ ]:
import logging

import amici
from amici.petab.petab_importer import *
from amici.petab.simulations import RDATAS
from amici.plotting import *
from petab.v2 import Problem

We load a PEtab problem from the [PEtab benchmark problem collection](https://github.com/Benchmarking-Initiative/Benchmark-Models-PEtab).
Currently, these are only available in PEtab 1.0 format. However, when loading them with `petab.v2.Problem.from_yaml`, they will be automatically converted to the PEtab 2.0 format.

In [ ]:
# Load a PEtab v2 problem
problem_id = "Boehm_JProteomeRes2014"
petab_yaml = f"https://benchmarking-initiative.github.io/Benchmark-Models-PEtab/tree/Benchmark-Models/{problem_id}/{problem_id}.yaml"
problem = Problem.from_yaml(petab_yaml)

print(problem)

First, we create an AMICI model from the PEtab v2 problem via `PetabImporter`. This will account for the observation model encoded in the PEtab problem, as well as for the different experimental conditions. This is important to keep in mind when using the model anything else than the PEtab-encoded experiments, which should generally be avoided. For the actual simulations, the easiest way is to use a `PetabSimulator`.


In [ ]:
importer = PetabImporter(problem, verbose=logging.INFO)
simulator = importer.create_simulator()

Now let's run the simulations:

In [ ]:
# simulate all conditions encoded in the PEtab problem for which there are measurements
#  using the nominal parameter values from the PEtab problem
result = simulator.simulate(problem.get_x_nominal_dict())
assert all(r.status == amici.AMICI_SUCCESS for r in result[RDATAS]), (
    "Simulation failed."
)
result

The returned dictionary contains the simulation results for all experimental conditions encoded in the PEtab problem in `result['rdatas']`.
Those are the same objects as for any other AMICI simulation using `amici.run_simulation`.
Additionally, the dictionary contains the `ExpData` instances used for the simulations in `result['edatas']`, which we will use below to visualize the PEtab-encoded measurements.
`result['llh']` and `result['sllh']` contain the aggregated log-likelihood value and its gradient, respectively, over all experimental conditions.
These can be used directly for parameter estimation. However, for parameter estimation, it is recommended to use the `pypesto` package that provides a full parameter estimation framework on top of AMICI and PEtab.

Now, let's have a look at the results of the first experimental condition:

In [ ]:
rdata = result["rdatas"][0]
edata = result["edatas"][0]
plot_observable_trajectories(rdata, model=simulator.model, edata=edata)

Simulation settings can be adjusted via the `PetabSimulator.solver` and `PetabSimulator.model` attributes, which are instances of `amici.Solver` and `amici.Solver`, respectively.

## Simulating individual conditions

It's also possible to simulate only specific experimental conditions encoded in the PEtab problem. The `ExperimentManager` takes care of creating `ExpData` the respective instances, and setting the condition-specific parameters, measurements, and initial states:

In [ ]:
model = importer.create_model()
# It's important to use the petab problem from the importer, as it was modified to encode the experimental conditions.
em = ExperimentManager(model=model, petab_problem=importer.petab_problem)

These are the experiments encoded in the PEtab problem:

In [ ]:
importer.petab_problem.experiments

We can now create an `ExpData` instance for any experiment, by passing the `petab.v2.Experiment` or its ID to `em.create_edata`, and simulate it as usual with AMICI:

In [ ]:
edata = em.create_edata(importer.petab_problem.experiments[-1])
edata

In [ ]:
rdata = model.simulate(edata=edata)
assert rdata.status == amici.AMICI_SUCCESS, "Simulation failed."
rdata

In [ ]:
plot_observable_trajectories(rdata, model=model, edata=edata)
plot_state_trajectories(rdata, model=model)

In [ ]:
rdata.xr.x.to_pandas()

The parameter values used for the simulation are the nominal values from the PEtab problem by default. You can, of course, also provide custom parameter values.
The parameter values can be updated using `em.apply_parameters({'parameter1': 0.1, ... })`.

That's it! You have successfully imported and simulated a PEtab v2 problem using AMICI.

Changing simulation settings, e.g., tolerances, sensitivity methods, etc., works as usual with AMICI models. Check out the other AMICI notebooks for more information.